<a href="https://colab.research.google.com/github/mohit821967/Shopify-store-Insights-Fetcher-Application/blob/main/Shopify_store_Insights_Fetcher_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install required packages
!pip install fastapi uvicorn aiohttp beautifulsoup4 pydantic requests pandas numpy
!pip install nest-asyncio python-multipart jinja2 python-dotenv lxml

# Step 2: Import all necessary libraries
import asyncio
import aiohttp
import json
import re
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional, Any, Union
from urllib.parse import urljoin, urlparse
import nest_asyncio
from bs4 import BeautifulSoup
import requests
from pydantic import BaseModel, validator
import warnings
warnings.filterwarnings('ignore')

# Enable nested event loops for Colab
nest_asyncio.apply()

print("✅ All dependencies installed and imported successfully!")


ERROR: Operation cancelled by user
✅ All dependencies installed and imported successfully!


In [3]:
# STEP 3: UPDATED DATA MODELS WITH PROPER VALIDATION
class Product(BaseModel):
    id: int = 0
    title: str = ""
    handle: str = ""
    description: Optional[str] = ""
    vendor: str = ""
    product_type: str = ""
    price: float = 0.0
    compare_at_price: Optional[float] = None
    available: bool = True
    tags: List[str] = []
    images: List[str] = []

    @validator('tags', pre=True)
    def validate_tags(cls, v):
        if isinstance(v, str):
            return [tag.strip() for tag in v.split(',') if tag.strip()]
        elif isinstance(v, list):
            return [str(tag).strip() for tag in v if tag]
        return []

    @validator('price', pre=True)
    def validate_price(cls, v):
        try:
            return float(v) if v is not None else 0.0
        except (ValueError, TypeError):
            return 0.0

    @validator('compare_at_price', pre=True)
    def validate_compare_at_price(cls, v):
        if v is None or v == '' or v == 'null':
            return None
        try:
            return float(v)
        except (ValueError, TypeError):
            return None

    class Config:
        arbitrary_types_allowed = True

class SocialHandle(BaseModel):
    platform: str
    url: str
    username: Optional[str] = None

class ContactInfo(BaseModel):
    emails: List[str] = []
    phone_numbers: List[str] = []
    address: Optional[str] = None

class FAQ(BaseModel):
    question: str
    answer: str
    category: Optional[str] = None

class BrandInsights(BaseModel):
    store_url: str
    store_name: str
    product_catalog: List[Product] = []
    hero_products: List[Product] = []
    privacy_policy: Optional[str] = None
    return_policy: Optional[str] = None
    refund_policy: Optional[str] = None
    faqs: List[FAQ] = []
    social_handles: List[SocialHandle] = []
    contact_info: ContactInfo = ContactInfo()
    about_brand: Optional[str] = None
    important_links: Dict[str, str] = {}
    scraped_at: datetime = datetime.now()

    class Config:
        arbitrary_types_allowed = True

print("✅ Data models defined successfully!")


✅ Data models defined successfully!


In [4]:

#STEP 4: UPDATED SCRAPING ENGINE WITH BUG FIXES
class ShopifyStoreScraper:
    def __init__(self):
        self.session = None
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

    async def __aenter__(self):
        connector = aiohttp.TCPConnector(limit=30, limit_per_host=10)
        self.session = aiohttp.ClientSession(
            headers=self.headers,
            timeout=aiohttp.ClientTimeout(total=30),
            connector=connector
        )
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.session:
            await self.session.close()

    async def scrape_store(self, store_url: str) -> BrandInsights:
        """Main scraping method that orchestrates all data collection"""
        print(f"🔍 Starting to scrape: {store_url}")

        # Normalize URL
        if not store_url.startswith('http'):
            store_url = f'https://{store_url}'

        try:
            # Initialize insights object
            insights = BrandInsights(
                store_url=store_url,
                store_name=self.extract_store_name(store_url)
            )

            # Scrape different components
            print("📦 Scraping product catalog...")
            insights.product_catalog = await self.scrape_products(store_url)

            print("🏠 Scraping hero products...")
            insights.hero_products = await self.scrape_hero_products(store_url)

            print("📜 Scraping policies...")
            policies = await self.scrape_policies(store_url)
            insights.privacy_policy = policies.get('privacy')
            insights.return_policy = policies.get('return')
            insights.refund_policy = policies.get('refund')

            print("❓ Scraping FAQs...")
            insights.faqs = await self.scrape_faqs(store_url)

            print("📱 Scraping social handles...")
            insights.social_handles = await self.scrape_social_handles(store_url)

            print("📞 Scraping contact info...")
            insights.contact_info = await self.scrape_contact_info(store_url)

            print("ℹ️ Scraping about brand...")
            insights.about_brand = await self.scrape_about_brand(store_url)

            print("🔗 Scraping important links...")
            insights.important_links = await self.scrape_important_links(store_url)

            print("✅ Scraping completed successfully!")
            return insights

        except Exception as e:
            print(f"❌ Error scraping store: {str(e)}")
            raise

    def extract_store_name(self, url: str) -> str:
        """Extract store name from URL"""
        parsed = urlparse(url)
        domain = parsed.netloc.replace('www.', '')
        return domain.split('.')[0].title()

    async def scrape_products(self, store_url: str) -> List[Product]:
        """Scrape product catalog using products.json endpoint with proper error handling"""
        products = []
        try:
            # Try products.json endpoint first
            products_url = f"{store_url.rstrip('/')}/products.json"
            async with self.session.get(products_url) as response:
                if response.status == 200:
                    data = await response.json()

                    for product_data in data.get('products', []):
                        try:
                            # Safely extract product information
                            product_id = product_data.get('id', 0)
                            title = product_data.get('title', '')
                            handle = product_data.get('handle', '')
                            description = self.clean_html(product_data.get('body_html', ''))
                            vendor = product_data.get('vendor', '')
                            product_type = product_data.get('product_type', '')
                            available = product_data.get('available', True)

                            # Handle tags properly - fix for the main error
                            tags = product_data.get('tags', [])
                            if isinstance(tags, str):
                                tags = [tag.strip() for tag in tags.split(',') if tag.strip()]
                            elif isinstance(tags, list):
                                tags = [str(tag).strip() for tag in tags if tag]
                            else:
                                tags = []

                            # Safely extract price information
                            price = 0.0
                            compare_at_price = None
                            variants = product_data.get('variants', [])

                            if variants and len(variants) > 0:
                                first_variant = variants[0]
                                try:
                                    price = float(first_variant.get('price', 0))
                                except (ValueError, TypeError):
                                    price = 0.0

                                compare_price_raw = first_variant.get('compare_at_price')
                                if compare_price_raw and compare_price_raw != 'null':
                                    try:
                                        compare_at_price = float(compare_price_raw)
                                    except (ValueError, TypeError):
                                        compare_at_price = None

                            # Extract images
                            images = []
                            for img in product_data.get('images', []):
                                if isinstance(img, dict) and 'src' in img:
                                    images.append(img['src'])
                                elif isinstance(img, str):
                                    images.append(img)

                            # Create product object
                            product = Product(
                                id=product_id,
                                title=title,
                                handle=handle,
                                description=description,
                                vendor=vendor,
                                product_type=product_type,
                                price=price,
                                compare_at_price=compare_at_price,
                                available=available,
                                tags=tags,
                                images=images
                            )
                            products.append(product)

                        except Exception as e:
                            print(f"⚠️ Error parsing product {product_data.get('title', 'Unknown')}: {e}")
                            continue

            print(f"✅ Found {len(products)} products")
            return products

        except Exception as e:
            print(f"❌ Error scraping products: {e}")
            return []

    async def scrape_hero_products(self, store_url: str) -> List[Product]:
        """Scrape hero products from homepage with better error handling"""
        try:
            async with self.session.get(store_url) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')

                    hero_products = []

                    # Multiple selectors to find product links
                    product_selectors = [
                        'a[href*="/products/"]',
                        '.product-card a',
                        '.product-item a',
                        '[data-product-handle]',
                        '.product-link',
                        '.product a'
                    ]

                    seen_handles = set()

                    for selector in product_selectors:
                        try:
                            links = soup.select(selector)[:8]  # Limit to first 8

                            for link in links:
                                href = link.get('href', '')
                                if '/products/' in href:
                                    # Extract product handle
                                    handle = href.split('/products/')[-1].split('?')[0].split('#')[0]

                                    if handle and handle not in seen_handles and len(handle) > 0:
                                        seen_handles.add(handle)

                                        # Try to get product data
                                        try:
                                            product_json_url = f"{store_url.rstrip('/')}/products/{handle}.json"
                                            async with self.session.get(product_json_url) as prod_response:
                                                if prod_response.status == 200:
                                                    prod_data = await prod_response.json()
                                                    product_info = prod_data.get('product', {})

                                                    # Use the same parsing logic as main products
                                                    tags = product_info.get('tags', [])
                                                    if isinstance(tags, str):
                                                        tags = [tag.strip() for tag in tags.split(',') if tag.strip()]
                                                    elif isinstance(tags, list):
                                                        tags = [str(tag).strip() for tag in tags if tag]
                                                    else:
                                                        tags = []

                                                    # Get price safely
                                                    price = 0.0
                                                    variants = product_info.get('variants', [])
                                                    if variants:
                                                        try:
                                                            price = float(variants[0].get('price', 0))
                                                        except (ValueError, TypeError):
                                                            price = 0.0

                                                    hero_product = Product(
                                                        id=product_info.get('id', 0),
                                                        title=product_info.get('title', ''),
                                                        handle=product_info.get('handle', handle),
                                                        description=self.clean_html(product_info.get('body_html', '')),
                                                        vendor=product_info.get('vendor', ''),
                                                        product_type=product_info.get('product_type', ''),
                                                        price=price,
                                                        available=product_info.get('available', True),
                                                        tags=tags,
                                                        images=[img.get('src', '') for img in product_info.get('images', []) if isinstance(img, dict) and 'src' in img]
                                                    )
                                                    hero_products.append(hero_product)

                                        except Exception as e:
                                            print(f"⚠️ Error fetching product {handle}: {e}")
                                            continue
                        except Exception as e:
                            print(f"⚠️ Error with selector {selector}: {e}")
                            continue

                    print(f"✅ Found {len(hero_products)} hero products")
                    return hero_products

        except Exception as e:
            print(f"❌ Error scraping hero products: {e}")
            return []

    async def scrape_policies(self, store_url: str) -> Dict[str, str]:
        """Scrape various policies with better error handling"""
        policies = {}
        policy_paths = {
            'privacy': ['/pages/privacy-policy', '/policies/privacy-policy', '/pages/privacy', '/privacy-policy', '/privacy'],
            'return': ['/pages/return-policy', '/policies/return-policy', '/pages/returns', '/return-policy', '/returns'],
            'refund': ['/pages/refund-policy', '/policies/refund-policy', '/pages/refunds', '/refund-policy', '/refunds']
        }

        for policy_type, paths in policy_paths.items():
            for path in paths:
                try:
                    policy_url = f"{store_url.rstrip('/')}{path}"
                    async with self.session.get(policy_url) as response:
                        if response.status == 200:
                            html = await response.text()
                            soup = BeautifulSoup(html, 'html.parser')

                            # Remove unwanted elements
                            for element in soup(['script', 'style', 'nav', 'header', 'footer']):
                                element.decompose()

                            # Find main content
                            main_content = soup.find('main') or soup.find('div', class_=re.compile(r'content|main|policy'))

                            if main_content:
                                text = main_content.get_text(separator=' ', strip=True)
                            else:
                                text = soup.get_text(separator=' ', strip=True)

                            cleaned_text = self.clean_text(text)

                            if len(cleaned_text) > 100:  # Only store if substantial content
                                policies[policy_type] = cleaned_text[:2000]  # Limit length
                                break

                except Exception as e:
                    print(f"⚠️ Error scraping {policy_type} policy from {path}: {e}")
                    continue

        return policies

    async def scrape_faqs(self, store_url: str) -> List[FAQ]:
        """Scrape FAQs with improved parsing"""
        faqs = []
        faq_paths = ['/pages/faq', '/pages/faqs', '/faq', '/faqs', '/pages/frequently-asked-questions', '/help']

        for path in faq_paths:
            try:
                faq_url = f"{store_url.rstrip('/')}{path}"
                async with self.session.get(faq_url) as response:
                    if response.status == 200:
                        html = await response.text()
                        soup = BeautifulSoup(html, 'html.parser')

                        # Look for different FAQ structures
                        faq_patterns = [
                            ('details', 'summary'),  # Details/summary structure
                            ('.faq-item', '.faq-question'),  # FAQ item structure
                            ('.accordion-item', '.accordion-header'),  # Accordion structure
                        ]

                        for container_selector, question_selector in faq_patterns:
                            containers = soup.select(container_selector)

                            for container in containers:
                                try:
                                    question_elem = container.select_one(question_selector)
                                    if question_elem:
                                        question = question_elem.get_text(strip=True)

                                        # Get answer - different methods
                                        answer = ""
                                        if container.name == 'details':
                                            answer = container.get_text(strip=True).replace(question, '', 1)
                                        else:
                                            answer_elem = container.select_one('.faq-answer, .accordion-content, .answer')
                                            if answer_elem:
                                                answer = answer_elem.get_text(strip=True)

                                        if question and answer and len(question) > 5 and len(answer) > 5:
                                            faq = FAQ(
                                                question=question[:300],
                                                answer=answer[:800],
                                                category="General"
                                            )
                                            faqs.append(faq)

                                except Exception as e:
                                    print(f"⚠️ Error parsing FAQ item: {e}")
                                    continue

                        if faqs:
                            break

            except Exception as e:
                print(f"⚠️ Error scraping FAQs from {path}: {e}")
                continue

        print(f"✅ Found {len(faqs)} FAQs")
        return faqs[:15]  # Limit to 15 FAQs

    async def scrape_social_handles(self, store_url: str) -> List[SocialHandle]:
        """Scrape social media handles with better detection"""
        social_handles = []

        try:
            async with self.session.get(store_url) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')

                    # Social media patterns
                    social_patterns = {
                        'instagram': [r'instagram\.com/([^/?\s]+)', r'instagr\.am/([^/?\s]+)'],
                        'facebook': [r'facebook\.com/([^/?\s]+)', r'fb\.com/([^/?\s]+)'],
                        'twitter': [r'twitter\.com/([^/?\s]+)', r'x\.com/([^/?\s]+)'],
                        'youtube': [r'youtube\.com/([^/?\s]+)', r'youtu\.be/([^/?\s]+)'],
                        'tiktok': [r'tiktok\.com/([^/?\s]+)'],
                        'linkedin': [r'linkedin\.com/([^/?\s]+)'],
                        'pinterest': [r'pinterest\.com/([^/?\s]+)'],
                        'snapchat': [r'snapchat\.com/([^/?\s]+)']
                    }

                    # Find all links
                    links = soup.find_all('a', href=True)

                    seen_platforms = set()

                    for link in links:
                        href = link.get('href', '')

                        for platform, patterns in social_patterns.items():
                            if platform not in seen_platforms:
                                for pattern in patterns:
                                    match = re.search(pattern, href, re.IGNORECASE)
                                    if match:
                                        username = match.group(1)
                                        # Clean username
                                        username = username.split('?')[0].split('#')[0].strip('/')

                                        if username and len(username) > 0:
                                            social_handle = SocialHandle(
                                                platform=platform,
                                                url=href,
                                                username=username
                                            )
                                            social_handles.append(social_handle)
                                            seen_platforms.add(platform)
                                            break

        except Exception as e:
            print(f"❌ Error scraping social handles: {e}")

        print(f"✅ Found {len(social_handles)} social handles")
        return social_handles

    async def scrape_contact_info(self, store_url: str) -> ContactInfo:
        """Scrape contact information with better extraction"""
        contact_info = ContactInfo()

        try:
            # Try contact page first, then homepage
            contact_urls = [
                f"{store_url.rstrip('/')}/pages/contact",
                f"{store_url.rstrip('/')}/contact",
                store_url
            ]

            html_content = ""
            for url in contact_urls:
                try:
                    async with self.session.get(url) as response:
                        if response.status == 200:
                            html_content = await response.text()
                            break
                except:
                    continue

            if html_content:
                # Improved email regex
                email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                emails = re.findall(email_pattern, html_content)

                # Improved phone regex
                phone_patterns = [
                    r'(\+?91[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',  # Indian format
                    r'(\+?1[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}',   # US format
                    r'(\+?44[-.\s]?)?\(?\d{3,4}\)?[-.\s]?\d{3,4}[-.\s]?\d{4}' # UK format
                ]

                phones = []
                for pattern in phone_patterns:
                    matches = re.findall(pattern, html_content)
                    phones.extend(matches)

                # Clean and deduplicate
                unique_emails = list(set([email.lower() for email in emails if '@' in email and '.' in email]))
                unique_phones = list(set([phone if isinstance(phone, str) else '-'.join(phone) for phone in phones]))

                contact_info.emails = unique_emails[:3]  # Limit to 3 emails
                contact_info.phone_numbers = unique_phones[:3]  # Limit to 3 phones

        except Exception as e:
            print(f"❌ Error scraping contact info: {e}")

        return contact_info

    async def scrape_about_brand(self, store_url: str) -> Optional[str]:
        """Scrape about brand information with better content extraction"""
        about_paths = ['/pages/about', '/pages/about-us', '/pages/our-story', '/about', '/about-us', '/our-story']

        for path in about_paths:
            try:
                about_url = f"{store_url.rstrip('/')}{path}"
                async with self.session.get(about_url) as response:
                    if response.status == 200:
                        html = await response.text()
                        soup = BeautifulSoup(html, 'html.parser')

                        # Remove unwanted elements
                        for element in soup(['script', 'style', 'nav', 'header', 'footer']):
                            element.decompose()

                        # Find main content areas
                        content_selectors = [
                            'main',
                            '.about-content',
                            '.content',
                            '.main-content',
                            '#main-content',
                            '.page-content'
                        ]

                        main_content = None
                        for selector in content_selectors:
                            main_content = soup.select_one(selector)
                            if main_content:
                                break

                        if not main_content:
                            main_content = soup.find('body')

                        if main_content:
                            text = main_content.get_text(separator=' ', strip=True)
                            cleaned_text = self.clean_text(text)

                            if len(cleaned_text) > 100:
                                return cleaned_text[:1500]  # Limit to 1500 characters

            except Exception as e:
                print(f"⚠️ Error scraping about from {path}: {e}")
                continue

        return None

    async def scrape_important_links(self, store_url: str) -> Dict[str, str]:
        """Scrape important links with better detection"""
        important_links = {}

        try:
            async with self.session.get(store_url) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')

                    # Important link keywords
                    important_keywords = {
                        'track': ['track', 'tracking', 'order tracking'],
                        'contact': ['contact', 'contact us', 'get in touch'],
                        'blog': ['blog', 'news', 'articles'],
                        'size': ['size', 'size guide', 'sizing'],
                        'shipping': ['shipping', 'delivery', 'shipping policy'],
                        'support': ['support', 'help', 'customer service'],
                        'returns': ['returns', 'return policy'],
                        'careers': ['careers', 'jobs', 'work with us']
                    }

                    links = soup.find_all('a', href=True)

                    for link in links:
                        href = link.get('href', '')
                        text = link.get_text(strip=True).lower()

                        if len(text) > 50:  # Skip very long texts
                            continue

                        for category, keywords in important_keywords.items():
                            if any(keyword in text for keyword in keywords):
                                if href.startswith('/'):
                                    href = f"{store_url.rstrip('/')}{href}"
                                elif not href.startswith('http'):
                                    href = f"{store_url.rstrip('/')}/{href}"

                                clean_text = text.title()
                                if clean_text not in important_links:
                                    important_links[clean_text] = href
                                break

        except Exception as e:
            print(f"❌ Error scraping important links: {e}")

        return important_links

    def clean_html(self, html_string: str) -> str:
        """Clean HTML content"""
        if not html_string:
            return ""
        soup = BeautifulSoup(html_string, 'html.parser')
        return soup.get_text(separator=' ', strip=True)

    def clean_text(self, text: str) -> str:
        """Clean and normalize text"""
        if not text:
            return ""
        # Remove extra whitespace and normalize
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', ' ', text)
        return text.strip()

print("✅ Updated scraping engine defined successfully!")


✅ Updated scraping engine defined successfully!


In [5]:
# STEP 5: MAIN APPLICATION INTERFACE

class ShopifyInsightsFetcher:
    def __init__(self):
        self.scraper = ShopifyStoreScraper()

    async def analyze_store(self, store_url: str) -> BrandInsights:
        """Analyze a single Shopify store"""
        async with self.scraper as scraper:
            return await scraper.scrape_store(store_url)

    def display_insights(self, insights: BrandInsights):
        """Display insights in a formatted way"""
        print("="*80)
        print(f"🏪 STORE ANALYSIS: {insights.store_name}")
        print(f"🌐 URL: {insights.store_url}")
        print(f"📅 Scraped: {insights.scraped_at}")
        print("="*80)

        print(f"\n📦 PRODUCT CATALOG ({len(insights.product_catalog)} products)")
        if insights.product_catalog:
            for i, product in enumerate(insights.product_catalog[:5]):  # Show first 5
                print(f"  {i+1}. {product.title}")
                print(f"     Price: ${product.price} | Vendor: {product.vendor}")
                print(f"     Type: {product.product_type} | Available: {product.available}")
                if product.tags:
                    print(f"     Tags: {', '.join(product.tags[:3])}")
                print()
            if len(insights.product_catalog) > 5:
                print(f"  ... and {len(insights.product_catalog) - 5} more products")

        print(f"\n🏠 HERO PRODUCTS ({len(insights.hero_products)} products)")
        if insights.hero_products:
            for i, product in enumerate(insights.hero_products):
                print(f"  {i+1}. {product.title} - ${product.price}")

        print(f"\n📱 SOCIAL HANDLES ({len(insights.social_handles)} handles)")
        for handle in insights.social_handles:
            print(f"  {handle.platform.title()}: {handle.url}")
            if handle.username:
                print(f"    Username: @{handle.username}")

        print(f"\n📞 CONTACT INFO")
        if insights.contact_info.emails:
            print(f"  📧 Emails: {', '.join(insights.contact_info.emails)}")
        if insights.contact_info.phone_numbers:
            print(f"  📱 Phones: {', '.join(insights.contact_info.phone_numbers)}")

        print(f"\n❓ FAQs ({len(insights.faqs)} found)")
        for i, faq in enumerate(insights.faqs[:3]):  # Show first 3
            print(f"  Q{i+1}: {faq.question}")
            print(f"  A{i+1}: {faq.answer[:150]}...")
            print()

        print(f"\n🔗 IMPORTANT LINKS ({len(insights.important_links)} found)")
        for name, url in list(insights.important_links.items())[:5]:
            print(f"  {name}: {url}")

        print(f"\n📜 POLICIES")
        if insights.privacy_policy:
            print(f"  ✅ Privacy Policy: {len(insights.privacy_policy)} characters")
        if insights.return_policy:
            print(f"  ✅ Return Policy: {len(insights.return_policy)} characters")
        if insights.refund_policy:
            print(f"  ✅ Refund Policy: {len(insights.refund_policy)} characters")

        if insights.about_brand:
            print(f"\nℹ️ ABOUT BRAND")
            print(f"  {insights.about_brand[:300]}...")

        print("="*80)

    def export_to_json(self, insights: BrandInsights, filename: str = None):
        """Export insights to JSON file"""
        if not filename:
            filename = f"{insights.store_name}_insights.json"

        # Convert to dict for JSON serialization
        insights_dict = insights.dict()

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(insights_dict, f, indent=2, ensure_ascii=False, default=str)

        print(f"📁 Insights exported to {filename}")
        return filename

    def create_summary_dataframe(self, insights_list: List[BrandInsights]) -> pd.DataFrame:
        """Create a summary DataFrame from multiple insights"""
        summary_data = []

        for insights in insights_list:
            summary_data.append({
                'Store Name': insights.store_name,
                'URL': insights.store_url,
                'Total Products': len(insights.product_catalog),
                'Hero Products': len(insights.hero_products),
                'Social Handles': len(insights.social_handles),
                'FAQs': len(insights.faqs),
                'Has Privacy Policy': bool(insights.privacy_policy),
                'Has Return Policy': bool(insights.return_policy),
                'Has About Section': bool(insights.about_brand),
                'Contact Emails': len(insights.contact_info.emails),
                'Contact Phones': len(insights.contact_info.phone_numbers),
                'Important Links': len(insights.important_links),
                'Scraped At': insights.scraped_at.strftime('%Y-%m-%d %H:%M:%S')
            })

        return pd.DataFrame(summary_data)

print("✅ Main application interface ready!")


✅ Main application interface ready!


In [6]:
# STEP 6: DEMO FUNCTIONS

# Initialize the fetcher
fetcher = ShopifyInsightsFetcher()

# Demo 1: Single store analysis
async def demo_single_store(store_url: str = "https://memy.co.in"):
    print("🚀 Demo: Single Store Analysis")
    print(f"Target: {store_url}")
    print("-" * 50)

    try:
        insights = await fetcher.analyze_store(store_url)
        fetcher.display_insights(insights)

        # Export to JSON
        json_file = fetcher.export_to_json(insights)

        return insights
    except Exception as e:
        print(f"❌ Error analyzing store: {e}")
        return None

# Demo 2: Multiple stores comparison
async def demo_multiple_stores(store_urls: List[str] = None):
    if store_urls is None:
        store_urls = [
            "https://memy.co.in",
            "https://hairoriginals.com"
        ]

    print("🚀 Demo: Multiple Stores Analysis")
    print(f"Analyzing {len(store_urls)} stores...")
    print("-" * 50)

    all_insights = []

    for i, url in enumerate(store_urls, 1):
        print(f"\n📍 [{i}/{len(store_urls)}] Analyzing: {url}")
        try:
            insights = await fetcher.analyze_store(url)
            all_insights.append(insights)
            print(f"✅ Successfully analyzed {insights.store_name}")
        except Exception as e:
            print(f"❌ Error analyzing {url}: {e}")

    # Create summary DataFrame
    if all_insights:
        summary_df = fetcher.create_summary_dataframe(all_insights)
        print("\n📊 SUMMARY COMPARISON")
        print("=" * 100)
        print(summary_df.to_string(index=False))

        # Export summary to CSV
        summary_df.to_csv('stores_comparison.csv', index=False)
        print(f"\n📁 Summary exported to stores_comparison.csv")

    return all_insights

# Demo 3: Quick analysis
async def quick_analyze(store_url: str):
    """Quick analysis with minimal output"""
    print(f"🔍 Quick analysis of: {store_url}")
    print("-" * 30)

    try:
        insights = await fetcher.analyze_store(store_url)

        print(f"\n📊 QUICK SUMMARY:")
        print(f"🏪 Store: {insights.store_name}")
        print(f"📦 Products: {len(insights.product_catalog)}")
        print(f"🏠 Hero Products: {len(insights.hero_products)}")
        print(f"📱 Social Handles: {len(insights.social_handles)}")
        print(f"❓ FAQs: {len(insights.faqs)}")
        print(f"📜 Policies: {sum([bool(insights.privacy_policy), bool(insights.return_policy), bool(insights.refund_policy)])}/3")
        print(f"🔗 Important Links: {len(insights.important_links)}")
        print(f"📞 Contact Info: {len(insights.contact_info.emails)} emails, {len(insights.contact_info.phone_numbers)} phones")

        return insights
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("✅ Demo functions ready!")


✅ Demo functions ready!


In [7]:
# STEP 7: RUN THE DEMO

# Method 1: Analyze a single store (memy.co.in)
print("🎯 STARTING SINGLE STORE ANALYSIS")
print("=" * 60)

insights = await demo_single_store("https://memy.co.in")

if insights:
    print("\n🎉 Analysis completed successfully!")
    print(f"📊 Results Summary:")
    print(f"   Products found: {len(insights.product_catalog)}")
    print(f"   Hero products: {len(insights.hero_products)}")
    print(f"   Social handles: {len(insights.social_handles)}")
    print(f"   FAQs: {len(insights.faqs)}")
    print(f"   Policies: {sum([bool(insights.privacy_policy), bool(insights.return_policy), bool(insights.refund_policy)])}")
else:
    print("❌ Analysis failed!")


🎯 STARTING SINGLE STORE ANALYSIS
🚀 Demo: Single Store Analysis
Target: https://memy.co.in
--------------------------------------------------
🔍 Starting to scrape: https://memy.co.in
📦 Scraping product catalog...
✅ Found 30 products
🏠 Scraping hero products...
✅ Found 7 hero products
📜 Scraping policies...
❓ Scraping FAQs...
✅ Found 0 FAQs
📱 Scraping social handles...
✅ Found 2 social handles
📞 Scraping contact info...
ℹ️ Scraping about brand...
🔗 Scraping important links...
✅ Scraping completed successfully!
🏪 STORE ANALYSIS: Memy
🌐 URL: https://memy.co.in
📅 Scraped: 2025-08-16 09:48:54.605912

📦 PRODUCT CATALOG (30 products)
  1. Royal Purple Cotton Printed Kurta Set with Dupatta
     Price: $1299.0 | Vendor: Me & My
     Type: 3pc Set | Available: True
     Tags: 3pc, 3PC_1 LOT, 3XL

  2. Frost Shell Off-White Cotton Flex Co-ord Set
     Price: $999.0 | Vendor: Me & My
     Type: Co-ord Set | Available: True
     Tags: best, CFCS_2 LOT, Co-ord set

  3. Pearl Mist Off-White Cotton Fl

In [8]:
# STEP 8: ANALYZE ANOTHER STORE
# You can analyze any other Shopify store by changing the URL
custom_store_url = "https://hairoriginals.com"  # Change this to any Shopify store

print(f"\n🎯 ANALYZING CUSTOM STORE: {custom_store_url}")
print("=" * 60)

custom_insights = await demo_single_store(custom_store_url)



🎯 ANALYZING CUSTOM STORE: https://hairoriginals.com
🚀 Demo: Single Store Analysis
Target: https://hairoriginals.com
--------------------------------------------------
🔍 Starting to scrape: https://hairoriginals.com
📦 Scraping product catalog...
✅ Found 30 products
🏠 Scraping hero products...
✅ Found 10 hero products
📜 Scraping policies...
❓ Scraping FAQs...
✅ Found 20 FAQs
📱 Scraping social handles...
✅ Found 3 social handles
📞 Scraping contact info...
ℹ️ Scraping about brand...
🔗 Scraping important links...
✅ Scraping completed successfully!
🏪 STORE ANALYSIS: Hairoriginals
🌐 URL: https://hairoriginals.com
📅 Scraped: 2025-08-16 09:48:54.605912

📦 PRODUCT CATALOG (30 products)
  1. Wavy Topper Hair Extensions For Women | Wavy & 100% Real Hair For Crown Area
     Price: $11399.0 | Vendor: HairOriginals
     Type:  | Available: True
     Tags: Hair Extension, Hair Originals

  2. Bow Clips
     Price: $200.0 | Vendor: HairOriginals
     Type:  | Available: True

  3. Clip-In Hair Streaks

In [9]:
# STEP 9: COMPARE MULTIPLE STORES
# Compare multiple stores
store_list = [
    "https://memy.co.in",
    "https://hairoriginals.com"
]

print("\n🎯 COMPARING MULTIPLE STORES")
print("=" * 60)

comparison_results = await demo_multiple_stores(store_list)

if comparison_results:
    print(f"\n🎉 Comparison completed! Analyzed {len(comparison_results)} stores successfully.")



🎯 COMPARING MULTIPLE STORES
🚀 Demo: Multiple Stores Analysis
Analyzing 2 stores...
--------------------------------------------------

📍 [1/2] Analyzing: https://memy.co.in
🔍 Starting to scrape: https://memy.co.in
📦 Scraping product catalog...
✅ Found 30 products
🏠 Scraping hero products...
✅ Found 7 hero products
📜 Scraping policies...
❓ Scraping FAQs...
✅ Found 0 FAQs
📱 Scraping social handles...
✅ Found 2 social handles
📞 Scraping contact info...
ℹ️ Scraping about brand...
🔗 Scraping important links...
✅ Scraping completed successfully!
✅ Successfully analyzed Memy

📍 [2/2] Analyzing: https://hairoriginals.com
🔍 Starting to scrape: https://hairoriginals.com
📦 Scraping product catalog...
✅ Found 30 products
🏠 Scraping hero products...
✅ Found 10 hero products
📜 Scraping policies...
❓ Scraping FAQs...
✅ Found 20 FAQs
📱 Scraping social handles...
✅ Found 3 social handles
📞 Scraping contact info...
ℹ️ Scraping about brand...
🔗 Scraping important links...
✅ Scraping completed successful

In [10]:
# STEP 10: EXPORT AND DOWNLOAD RESULTS

# Function to display files created
import os

def show_generated_files():
    print("\n📁 GENERATED FILES:")
    print("=" * 40)

    json_files = [f for f in os.listdir('.') if f.endswith('_insights.json')]
    csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

    print("📄 JSON Files (Store Insights):")
    for file in json_files:
        size = os.path.getsize(file) / 1024  # Size in KB
        print(f"   {file} ({size:.1f} KB)")

    print("\n📊 CSV Files (Comparisons):")
    for file in csv_files:
        size = os.path.getsize(file) / 1024  # Size in KB
        print(f"   {file} ({size:.1f} KB)")

    print(f"\n💾 Total files generated: {len(json_files) + len(csv_files)}")

# Show all generated files
show_generated_files()



📁 GENERATED FILES:
📄 JSON Files (Store Insights):
   Hairoriginals_insights.json (119.1 KB)
   Memy_insights.json (95.6 KB)

📊 CSV Files (Comparisons):
   stores_comparison.csv (0.3 KB)

💾 Total files generated: 3


In [11]:
pdf = pd.read_csv('stores_comparison.csv')

In [12]:
print(pdf)

      Store Name                        URL  Total Products  Hero Products  \
0           Memy         https://memy.co.in              30              7   
1  Hairoriginals  https://hairoriginals.com              30             10   

   Social Handles  FAQs  Has Privacy Policy  Has Return Policy  \
0               2     0                True              False   
1               3    15                True              False   

   Has About Section  Contact Emails  Contact Phones  Important Links  \
0               True               2               3                4   
1               True               1               3                5   

            Scraped At  
0  2025-08-16 09:48:54  
1  2025-08-16 09:48:54  
